# Pipeline

## 서론

pipeline은 전처리, 모델링 과정을 쉽게 처리할 수 있도록 설계하는 것이다.  pipeline을 사용한다면 다음과 같은 이점을 얻을 수 있다.

- Cleaner code : pipeline을 사용한다면, train, valid 데이터세트를 하나하나 전처리 할 필요가 없다. 
- Fewer Bugs : 버그가 더 적다.
- Easier to Productionize : 더욱 생산적이다.
- More Options for Model Validation : 예제를 통해 확인해볼 것임.

## 예제

In [1]:
# 라이브러리 불러오기
import pandas as pd
from sklearn.model_selection import train_test_split

# 데이터 불러오기
data = pd.read_csv('data/melb_data.csv')

X = data.drop('Price', axis = 1)
y = data['Price']

# train, valid 나누기
X_train_full, X_valid_full, y_train, y_valid  = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Cardinality가 상대적으로 낮은 컬럼 선택
categorical_cols = [cname for cname in X_train_full.columns if X_train_full[cname].nunique() < 10 and X_train_full[cname].dtype == "object"]

# numeric 컬럼 선택
numeric_cols = [cname for cname in X_train_full.columns if X_train_full[cname].dtype in ['int64', 'float64']]

# 선택된 컬럼들로 X_train, X_valid 구성
my_cols = categorical_cols + numeric_cols
X_train = X_train_full[my_cols].copy()
X_valid = X_valid_full[my_cols].copy()

In [2]:
X_train.head()

,Type,Method,Regionname,Rooms,Distance,Postcode,Bedroom2,Bathroom,Car,Landsize,BuildingArea,YearBuilt,Lattitude,Longtitude,Propertycount
12167,u,S,Southern Metropolitan,1,5.0,3182.0,1.0,1.0,1.0,0.0,NaN,1940.0,-37.85984,144.9867,13240.0
6524,h,SA,Western Metropolitan,2,8.0,3016.0,2.0,2.0,1.0,193.0,NaN,NaN,-37.85800,144.9005,6380.0
8413,h,S,Western Metropolitan,3,12.6,3020.0,3.0,1.0,1.0,555.0,NaN,NaN,-37.79880,144.8220,3755.0
2919,u,SP,Northern Metropolitan,3,13.0,3046.0,3.0,1.0,1.0,265.0,NaN,1995.0,-37.70830,144.9158,8870.0
6043,h,S,Western Metropolitan,3,13.3,3020.0,3.0,1.0,2.0,673.0,673.0,1970.0,-37.76230,144.8272,4217.0


3가지 스텝으로 구성된 pipeline 코드를 만들어 볼 것이다.

### Step 1. Define Preprocessing Steps

numerical, categorical 변수를 함께 처리하기 위해 ColumnTransformer를 사용한다.

In [3]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder

# numerical 대체
numerical_transformer = SimpleImputer(strategy = 'constant')

# categorical 대체
categorical_transformer = Pipeline([('impuer', SimpleImputer(strategy = 'most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Bundle
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)])

### Step 2. Define the Model


In [4]:
from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor(n_estimators = 100, random_state = 0)

### Step 3. Create and Evaluate the Pipeline

pipelin을 사용한다면 우리는 train 데이터에 전처리와 모델 학습을 한 줄의 코드로 적용할 수 있다.

In [5]:
from sklearn.metrics import mean_absolute_error

# 전처리와 모델 학습을 bundle 처리
pipeline = Pipeline([('preprocessor', preprocessor),
                    ('model', model)])

# 모델 학습 및 예측
pipeline.fit(X_train, y_train)
preds = pipeline.predict(X_valid)

# 모델 평가
score = mean_absolute_error(y_valid, preds)
print('MAE', score)

MAE 160679.18917034855


## 결론
pipeline을 사용하면 코드도 단축되고, 에러도 피할 수 있다. 또한 정교한 전처리 작업의 워크플로우도 간결하게 만들어준다. 

## Reference

[Intermediate Machine Learning(Kaggle)](https://www.kaggle.com/alexisbcook/pipelines)